In [1]:
import os

import pandas as pd
import geopandas as gpd

from shapely.geometry import box

import planetary_computer as pc

import find_scenes as fs
from rasterio import CRS

import pystac_client

In [2]:
itemids = pd.read_csv(os.path.join(os.getcwd(), 'coastal_scenes_ids.csv')).itemid

In [3]:
all_boxes = []

for itemid in itemids:
    # ----------------------------------------------------------
    # search scene using id
    URL = "https://planetarycomputer.microsoft.com/api/stac/v1"
    catalog = pystac_client.Client.open(URL)

    search = catalog.search(
        collections=["naip"],
        ids = itemid)

    # return 1st item in search (assumes itemid IS associaed to some item)
    item = list(search.get_items())[0]

    # ----------------------------------------------------------
    # make scene's bounding box into a geodataframe with crs 4326
    corners = item.properties['proj:bbox']
    item_box = box(corners[0], corners[1], corners[2], corners[3])

    box_df = gpd.GeoDataFrame({'geometry': [item_box],
                               'id' : [item.id],
                               'year' : [item.properties['naip:year']] }, 
                                crs=CRS.from_epsg(item.properties['proj:epsg']))

    box_df = box_df.to_crs(CRS.from_epsg(4326))

    # ----------------------------------------------------------

    all_boxes.append(box_df)

In [7]:
all_boxes_df = pd.concat(all_boxes, ignore_index=True)
all_boxes_df

,geometry,id,year
0,"POLYGON ((-120.55919 34.49584, -120.55717 34.5...",ca_m_3412028_sw_10_1_20120519_20120730,2012
1,"POLYGON ((-120.49669 34.49583, -120.49461 34.5...",ca_m_3412028_se_10_1_20120519_20120730,2012
2,"POLYGON ((-120.55922 34.55840, -120.55719 34.6...",ca_m_3412028_nw_10_1_20120519_20120730,2012
3,"POLYGON ((-120.49667 34.55831, -120.49459 34.6...",ca_m_3412028_ne_10_1_20120519_20120730,2012
4,"POLYGON ((-120.62169 34.49590, -120.61972 34.5...",ca_m_3412027_se_10_1_20120519_20120730,2012
...,...,...,...
139,"POLYGON ((-119.62080 34.37327, -119.62291 34.4...",ca_m_3411935_se_11_060_20200521,2020
140,"POLYGON ((-119.80815 34.37328, -119.81041 34.4...",ca_m_3411934_sw_11_060_20200521,2020
141,"POLYGON ((-119.74572 34.37329, -119.74793 34.4...",ca_m_3411934_se_11_060_20200521,2020
142,"POLYGON ((-119.93306 34.37332, -119.93542 34.4...",ca_m_3411933_sw_11_060_20200521,2020


In [10]:
root = os.path.join(os.getcwd(), 'naip_scenes_footprints')
if os.path.exists(root) == False:
    os.mkdir(root)
all_boxes_df.to_file(os.path.join(root,'naip_scenes_footprints.shp'))